In [1]:
# !pip3 --version
#!pip install --upgrade pip

# python --version
# !pip install autocorrect

In [2]:
#imports 
import re
import pandas as pd
import os
import glob
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
sns.set_style('darkgrid')
from collections import Counter
from autocorrect import Speller


import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from io import StringIO
%matplotlib inline
import missingno as msno
import category_encoders as ce
import folium
from folium import Choropleth
#from folium.plugins import MarkerCluster
#from folium import plugins
from IPython.display import display
#from shapely import wkt


In [3]:
#!pip list

In [4]:

#to load all the sheets present in the xslx file 
# Path to the Excel file
file_path = '/Users/salu/Downloads/QVI_transaction_data.xlsx'

# Load the Excel file object
xls = pd.ExcelFile(file_path)

# Print all sheet names
print(xls.sheet_names)


['in']


In [5]:
# directory path for transaction file
directory_path_transaction = '/Users/salu/Downloads/' 
# pandas dataframe 
transaction_df =pd.read_excel(directory_path_transaction + 'QVI_transaction_data.xlsx')

In [6]:
transaction_df

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...
264831,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [7]:
# directory path for purchase behaviour 
directory_path_purchase = '/Users/salu/Downloads/' 
# pandas dataframe 
purchase_behav_df =pd.read_csv(directory_path_transaction + 'QVI_purchase_behaviour.csv')


In [8]:
purchase_behav_df

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


# **Transaction Data**

In [9]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


In [10]:
transaction_df.shape

(264836, 8)

In [11]:
# transaction_df.describe()

In [12]:
transaction_df.nunique()

DATE                 364
STORE_NBR            272
LYLTY_CARD_NBR     72637
TXN_ID            263127
PROD_NBR             114
PROD_NAME            114
PROD_QTY               6
TOT_SALES            112
dtype: int64

In [13]:
transaction_df.isna().sum()

DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

In [14]:
transaction_df.head(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
5,43604,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
6,43601,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,5.7
7,43601,4,4196,3539,24,Grain Waves Sweet Chilli 210g,1,3.6
8,43332,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,3.9
9,43330,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,2,7.2


In [15]:
transaction_df.tail(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
264826,43549,272,272194,269908,75,Cobs Popd Sea Salt Chips 110g,2,7.6
264827,43340,272,272197,269911,104,Infuzions Thai SweetChili PotatoMix 110g,2,7.6
264828,43308,272,272236,269974,68,Pringles Chicken Salt Crips 134g,2,7.4
264829,43540,272,272236,269976,49,Infuzions SourCream&Herbs Veg Strws 110g,2,7.6
264830,43416,272,272319,270087,44,Thins Chips Light& Tangy 175g,2,6.6
264831,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8
264835,43365,272,272380,270189,74,Tostitos Splash Of Lime 175g,2,8.8


In [16]:
transaction_df1 = transaction_df.copy()


In [17]:
print(transaction_df1.dtypes)

DATE                int64
STORE_NBR           int64
LYLTY_CARD_NBR      int64
TXN_ID              int64
PROD_NBR            int64
PROD_NAME          object
PROD_QTY            int64
TOT_SALES         float64
dtype: object


In [18]:
# the format of date needs to be changed in the dataframe 
# The number 43549 means the number of days since January 1, 1900 (Excel’s base date system on Windows)
transaction_df1['DATE'] = pd.to_datetime(transaction_df1['DATE'], origin='1899-12-30', unit='D')



In [19]:
transaction_df1.head(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
5,2019-05-19,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
6,2019-05-16,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,5.7
7,2019-05-16,4,4196,3539,24,Grain Waves Sweet Chilli 210g,1,3.6
8,2018-08-20,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,3.9
9,2018-08-18,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,2,7.2


In [20]:
transaction_df1.tail()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
264831,2019-03-09,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,2018-08-13,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,2018-11-06,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,2018-12-27,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8
264835,2018-09-22,272,272380,270189,74,Tostitos Splash Of Lime 175g,2,8.8


In [21]:
transaction_df1['PROD_NAME'].nunique()

114

In [22]:
product_name = transaction_df1['PROD_NAME'].unique()

In [36]:
product_name

,Products
0,Natural Chip Compny SeaSalt175g
1,CCs Nacho Cheese 175g
2,Smiths Crinkle Cut Chips Chicken 170g
3,Smiths Chip Thinly S/Cream&Onion 175g
4,Kettle Tortilla ChpsHny&Jlpno Chili 150g
...,...
109,Red Rock Deli Chikn&Garlic Aioli 150g
110,RRD SR Slow Rst Pork Belly 150g
111,RRD Pc Sea Salt 165g
112,Smith Crinkle Cut Bolognese 150g


In [24]:
# product_name_corrected = [map(spell, l) for l in product_name]


In [25]:
# print(product_name_corrected)

In [26]:
# splitting each string in the prod_name column to identify the products other than chips 
# 2. Split into individual words
all_words = []
for name in product_name:
    words = name.split()
    # remove digits and special characters , using regular expression and is placed into all_words
    clean_words = [w.lower() for w in words if re.match(r'^[a-zA-Z]+$', w)]
    all_words.extend(clean_words)

# 3. Count word frequencies
word_freq = Counter(all_words)

# 4. View most common words
print(word_freq.most_common(20))

[('chips', 21), ('smiths', 16), ('crinkle', 14), ('cut', 14), ('kettle', 13), ('cheese', 12), ('salt', 12), ('original', 10), ('chip', 9), ('salsa', 9), ('doritos', 9), ('corn', 8), ('pringles', 8), ('rrd', 8), ('chicken', 7), ('ww', 7), ('sour', 6), ('sea', 6), ('thinly', 5), ('vinegar', 5)]


In [27]:
type(all_words)

list

In [28]:
if "doritos" in all_words:
    print(all_words.count('doritos'))



9


In [29]:
nonchip_words = []
for word in all_words:
    if "chutney" in word:
        nonchip_words.append(word)
print(nonchip_words)

[]


In [30]:
product_name = pd.DataFrame(product_name)


In [31]:
# type(product_name) set the column name to avoid confusion

In [32]:
product_name.rename(columns={0: 'Products'}, inplace=True)

In [41]:
pd.set_option("display.max_rows", None)
product_name

,Products
0,Natural Chip Compny SeaSalt175g
1,CCs Nacho Cheese 175g
2,Smiths Crinkle Cut Chips Chicken 170g
3,Smiths Chip Thinly S/Cream&Onion 175g
4,Kettle Tortilla ChpsHny&Jlpno Chili 150g
5,Old El Paso Salsa Dip Tomato Mild 300g
6,Smiths Crinkle Chips Salt & Vinegar 330g
7,Grain Waves Sweet Chilli 210g
8,Doritos Corn Chip Mexican Jalapeno 150g
9,Grain Waves Sour Cream&Chives 210G


In [42]:
non_chip_terms = ["salsa", "dip", "chutney", "papadums", "aioli", "spread", "jam"]
chip_terms = ["chips", "chip", "doritos","dorito","cut","crinkle cut", "pringles", "pringle"]

In [35]:
# Dropping the spell checker its doing more harm than good perfectly good words are getting butchered
# from autocorrect import Speller
# configured to operate in English
# spell = Speller(lang='en')

# # List of product names
# # product_name

# # Correct spelling word-by-word
# corrected_names = []
# for name in product_name:
#     words = name.split()  # Split name into individual words
#     corrected_words = [spell(word) for word in words]  # Correct each word
#     corrected_name = ' '.join(corrected_words)  # Rejoin corrected words
#     corrected_names.append(corrected_name)

# # Show results
# for old, new in zip(product_name, corrected_names):
#     print(f"{old}  ➝  {new}")
